In [1]:
import os
import math
from PIL import Image
import random

In [2]:
def extract_features(image_path, size=(8, 8)):
    image = Image.open(image_path).convert('L')  # ubah ke grayscale
    image = image.resize(size)
    pixels = list(image.getdata())  # list of pixel values
    return pixels

In [3]:
def load_dataset(dataset_path):
    data = []
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        if not os.path.isdir(label_path): continue
        for filename in os.listdir(label_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(label_path, filename)
                features = extract_features(image_path)
                data.append({'label': label, 'features': features})
    return data

In [4]:
def split_data(data, split_ratio=0.8):
    random.shuffle(data)
    split_point = int(len(data) * split_ratio)
    return data[:split_point], data[split_point:]

In [5]:
def mean(values):
    return sum(values) / len(values)

def stddev(values, mean_val):
    variance = sum((x - mean_val) ** 2 for x in values) / len(values)
    return math.sqrt(variance)

In [6]:
def summarize_by_class(dataset):
    summaries = {}
    separated = {}

    for row in dataset:
        label = row['label']
        if label not in separated:
            separated[label] = []
        separated[label].append(row['features'])

    for label, features_list in separated.items():
        summaries[label] = []
        for i in range(len(features_list[0])):
            col = [row[i] for row in features_list]
            mean_val = mean(col)
            std_val = stddev(col, mean_val)
            summaries[label].append((mean_val, std_val))
    return summaries

In [7]:
def gaussian_prob(x, mean, std):
    if std == 0:
        return 1 if x == mean else 0
    exp = math.exp(-((x - mean) ** 2) / (2 * std ** 2))
    return (1 / (math.sqrt(2 * math.pi) * std)) * exp

In [8]:
def calculate_class_probabilities(summaries, input_vector):
    probabilities = {}
    for label, class_summaries in summaries.items():
        probabilities[label] = 1
        for i in range(len(class_summaries)):
            mean, std = class_summaries[i]
            x = input_vector[i]
            prob = gaussian_prob(x, mean, std)
            probabilities[label] *= prob
    return probabilities

def predict(summaries, input_vector):
    probs = calculate_class_probabilities(summaries, input_vector)
    return max(probs, key=probs.get)

In [9]:
if __name__ == "__main__":
    dataset_path = "dataset"
    data = load_dataset(dataset_path)

    if len(data) < 2:
        print("Dataset terlalu sedikit ngab. Tambahin beberapa gambar lagi.")
        exit()

    train_data, test_data = split_data(data)
    model = summarize_by_class(train_data)

    print(f"Jumlah data train: {len(train_data)}, test: {len(test_data)}\n")
    benar = 0
    log_lines = []

    for i, test in enumerate(test_data):
        pred = predict(model, test['features'])
        actual = test['label']
        if pred == actual:
            benar += 1
        line = f"[{i+1}] Asli: {actual:10s} → Prediksi: {pred:10s}"
        print(line)
        log_lines.append(line)

    akurasi = benar / len(test_data) * 100
    akurasi_line = f"\n🎯 Akurasi: {akurasi:.2f}%"
    print(akurasi_line)
    log_lines.append(akurasi_line)

    # Simpan ke file hasil_prediksi.txt
    with open("hasil_prediksi.txt", "w", encoding="utf-8") as f:
        for line in log_lines:
            f.write(line + "\n")

Jumlah data train: 211, test: 53

[1] Asli: sedih      → Prediksi: sedih     
[2] Asli: marah      → Prediksi: senang    
[3] Asli: sedih      → Prediksi: marah     
[4] Asli: marah      → Prediksi: marah     
[5] Asli: marah      → Prediksi: marah     
[6] Asli: sedih      → Prediksi: sedih     
[7] Asli: senang     → Prediksi: marah     
[8] Asli: marah      → Prediksi: sedih     
[9] Asli: sedih      → Prediksi: sedih     
[10] Asli: marah      → Prediksi: senang    
[11] Asli: senang     → Prediksi: senang    
[12] Asli: sedih      → Prediksi: sedih     
[13] Asli: marah      → Prediksi: senang    
[14] Asli: senang     → Prediksi: senang    
[15] Asli: senang     → Prediksi: sedih     
[16] Asli: senang     → Prediksi: senang    
[17] Asli: senang     → Prediksi: senang    
[18] Asli: sedih      → Prediksi: senang    
[19] Asli: senang     → Prediksi: senang    
[20] Asli: sedih      → Prediksi: sedih     
[21] Asli: marah      → Prediksi: marah     
[22] Asli: sedih      → Predik